As data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze. This chapter focuese on tools to help combine, join, and rearrange data.

In [1]:
import pandas as pd
import numpy as np

## Hierarchial indexing
*__Hierarchial Indexing__* is an important feature of pandas that enables you to have multiple(two or more) index levels on an axis. Somewhat abstraclty, it provides a way for you to work with higher dimensional data in a lower dimensional form.

In [2]:
data = pd.Series(np.random.randn(9), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                                            [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.307289
   2   -0.614461
   3   -0.226449
b  1   -1.103147
   3   -0.093489
c  1    0.590916
   2   -0.842094
d  2   -0.033315
   3    0.726151
dtype: float64

Above output is the prettified view of a Series with a __MultiIndex__ as its index

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With hierarchically indexed object, so-called __partial__ indexing is possible, enabling you to concisely select subsets of the data

In [4]:
data['b']

1   -1.103147
3   -0.093489
dtype: float64

In [5]:
data['b':'c']

b  1   -1.103147
   3   -0.093489
c  1    0.590916
   2   -0.842094
dtype: float64

In [6]:
data.loc[['b','d']]

b  1   -1.103147
   3   -0.093489
d  2   -0.033315
   3    0.726151
dtype: float64

In [7]:
#selection is even possible from an "inner" level
data.loc[:, 2]

a   -0.614461
c   -0.842094
d   -0.033315
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations like forming a pivot table. For example, you could <u>rearrange the data into a DF using __unstack__ method</u>

In [8]:
data.unstack()

,1,2,3
a,0.307289,-0.614461,-0.226449
b,-1.103147,NaN,-0.093489
c,0.590916,-0.842094,NaN
d,NaN,-0.033315,0.726151


In [9]:
#inverse operation of unstack is stack
data.unstack().stack()

a  1    0.307289
   2   -0.614461
   3   -0.226449
b  1   -1.103147
   3   -0.093489
c  1    0.590916
   2   -0.842094
d  2   -0.033315
   3    0.726151
dtype: float64

In [10]:
#with DF, either axis can have a hierarchical index
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
frame['Ohio']['Green']

key1  key2
a     1       0
      2       3
b     1       6
      2       9
Name: Green, dtype: int64

A __MultiIndex__ can be created by itself and then reused; the columns in the preceding DataFrame with level names could be created

In [14]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels
- __swaplevel__ takes two level numbers or names and returns a new object with the levels interchanged

In [15]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [16]:
#sort_index sorts the data using the values in a single value
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level—that is, the result of calling __sort_index(level=0) or sort_index().__

### Summary Statistics by Level
- Aggregate by level on either the rows or columns

In [18]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns
- Use one or more columns from a DataFrame as the row index
- Move the row index into the DataFrame's columns

In [21]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1), 
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


__set_index__ function will create a new DF using one or more of its columns as the index

In [22]:
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
#by default the columns are removed from DF, though you can leave them in
frame.set_index(['c','d'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [24]:
#reverse of set_index is reset_index as hierarchial index levels are moved into the columns
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combining and Merging Datasets
Data contained in pandas objects can be combined together in a number of ways:
- __pandas.merge__ connects rows in DF based on one or more keys. This is similar to __database join operations__.
- __pandas.concat__ concatenates or "stacks together objcts along an axis.
- __combine_first__ instance method enables splicing together overlapping data to fill in missing values in one object with values from another

### Database-Style DataFrame Joins
- __Merge or join__ operations combine datasets by linking rows using one or more *keys*

In [25]:
#example of many-to-one join as df1 has multiple rows labelled a and b wheres df2 has only one row for each value in key column
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [26]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [27]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If columns to join is not specified then __merge__ uses the overlapping column names as the keys. But it's a good practive to specify explicity

In [28]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [29]:
#different column names in each object
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')   #merging by specifying the key (column names) separately

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


All the above __merge__ code does a __inner__ joins; the keys in the result are the intersection or the common set found in both tables.<br>
Other joins such as __left, right, and other__ can also be performed.

In [30]:
pd.merge(df1, df2,  how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


#### Table: Different join types with how argument
| Option | Behavior |
| :----- | :------- |
| 'inner' | Use only the key combinations observed in both tables |
| 'left' | Use all key combinations found in the left table |
| 'right' | Use all key combinations found in the right table |
| 'output' | Use all key combinations observed in both tables together |

__Many-to-Many__ merges have well-defined, though not necessarily intuitive, behavior

In [31]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [33]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


__Many-to-many join form the Cartesian product of the rows. The join method only affects the distint key values appearing in the result__

In [34]:
#join method only affects the distinct key values appearing in the result
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [35]:
#to merge with multiple keys, pass a list of columns names
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


__merge__ has __suffixes__ option for specifying strings to append to overlapping names in the left and right DF objects

In [36]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [37]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### Table: merge function arguments
| Argument | Description |
| :------- | :---------- |
| left | DataFrame to be merged on the left side. |
| right | DataFrame to be merged on the right side. |
| how | One of 'inner', 'outer', 'left', or 'right'; defaults to 'inner'. |
| on | Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys given, will use the intersection of the column names in left and right as the join keys. |
| left_on | Columns in left DataFrame to use as join keys. |
| right_on | Analogous to left_on for left DataFrame. |
| left_index | Use row index in left as its join key (or keys, if a MultiIndex). |
| right_index | Analogous to left_index. |
| sort | Sort merged data lexicographically by join keys; True by default (disable to get better performance in some cases on large datasets). |
| suffixes | Tuple of string values to append to column names in case of overlap; defaults to ('\_x', '\_y') (e.g., if 'data' in both DataFrame objects, would appear as 'data_x' and 'data_y' in result). |
| copy | If False, avoid copying data into resulting data structure in some exceptional cases; by default always copies. |
| indicator | Adds a special column \_merge that indicates the source of each row; values will be 'left_only', 'right_only', or 'both' based on the origin of the joined data in each row. |

### Merging on Index
Sometimes, the merge key(s) in a DF will be found in its index. In this case, you can case __left_index=True__ or __right_index=True__ (or both) to indicate that the index should be used as the merge key

In [38]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [39]:
right1

,group_val
a,3.5
b,7.0


In [40]:
#intersect the join keys
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [41]:
#form the union of them with an outer join
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [42]:
#in hierarchically indexed data joining on index is implicityl a multiple-key merge
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio','Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
                                                            [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [43]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [44]:
#use multiple columns to merge on as a list
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [45]:
#handling of duplicate index values with outer join
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [46]:
#using indexes on both sides of the merge
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [47]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [48]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DF has a convenient __join__ instance for merging by index. It can also be used to combing together many DF objects having the same or similar indexes but non-overlapping columns.

In [49]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DF's __join__ method performs a left join on the join keys, exactly preserving the left frame's row index. It also supports joining the index of the passed DF on one of the columns of the calling DF

In [50]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [51]:
#can pass a list of DFs to join for simple index-on-index merges
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [52]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [53]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis
- Data combination operation referred to interchangeably as concatenation, binding, or stacking
- NumPy's __concatenate__ function can do this with NumPy arrays

In [54]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [55]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas objects such as Series and DataFrame, having labeled axes enable you to further generalize array concatenation. In particular, you have number of additional things to think about:
- If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the shared values (the intersection)?
- Do the concatenated chunks of data need to be identifiable in the resulting object?
- Does the "concatenation axis" contain data that needs to be preserved? In many cases, the default integer labels in a DF are best discarded during concatenation
<br>
<br>
The __concat__ function in pandas provides a consistent way to address each of these concerns.

In [56]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [57]:
#calling this method with pandas objects in alist glues together the values and indexes
pd.concat([s1, s2, s3]) #concatentaion at axis=0 by default

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [58]:
# concatenation with columns and the result be DF
pd.concat([s1,s2,s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In above case there is no overlap on the other axis, which is the sorted union (the __'outer'__ join)

In [59]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [60]:
pd.concat([s1,s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [61]:
#intersect series by passing join='inner'
pd.concat([s1,s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


__TO create a hierarchical index on the concatenation axis use the *keys* argument__

In [62]:
result = pd.concat([s1,s1,s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [63]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [64]:
#in the case of axis=1, the keys become the DF column headers
pd.concat([s1,s2,s3],axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [65]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [66]:
df2

,three,four
a,5,6
c,7,8


In [67]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [68]:
#passing dict of objects instead of a list, dict's key will be used for the keys option
pd.concat({'level1': df1, 'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [69]:
#name the created axis levels with the names argument
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [70]:
#row index of a DF does not contain any relevant data
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,0.654583,-1.021116,-0.542423,-0.296996
1,0.251437,-0.187479,1.035626,1.221486
2,0.136235,0.438647,-0.735153,1.375492


In [71]:
df2

,b,d,a
0,-0.091702,0.085596,0.436811
1,0.752159,-0.847553,-1.089164


In [72]:
pd.concat([df1, df2])

,a,b,c,d
0,0.654583,-1.021116,-0.542423,-0.296996
1,0.251437,-0.187479,1.035626,1.221486
2,0.136235,0.438647,-0.735153,1.375492
0,0.436811,-0.091702,NaN,0.085596
1,-1.089164,0.752159,NaN,-0.847553


In [73]:
pd.concat([df1, df2], ignore_index=True) #do not preserve indexes along concatenation axis

,a,b,c,d
0,0.654583,-1.021116,-0.542423,-0.296996
1,0.251437,-0.187479,1.035626,1.221486
2,0.136235,0.438647,-0.735153,1.375492
3,0.436811,-0.091702,NaN,0.085596
4,-1.089164,0.752159,NaN,-0.847553


#### Table: concat function arguments
| Argument | Description |
| :------- | :---------- |
| objs | List or dict of pandas objects to be concatenated; this is the only required argument |
| axis | Axis to concatenate along; defaults to 0 (along rows) |
| join | Either __'inner' or 'outer' ('outer' by default)__; whether to intersection (inner) or union (outer) together indexes along the other axes |
| keys | Values to associate with objects being concatenated, forming a hierarchical index along the concatenation axis; can either be a list or array of arbitrary values, an array of tuples, or a list of arrays (if multiple-level arrays passed in levels) |
| levels | Specific indexes to use as hierarchical index level or levels if keys passed |
| names | Names for created hierarchical levels if keys and/or levels passed |
| verify_integrity | Check new axis in concatenated object for duplicates and raise exception if so; by default (__False__) allows duplicates |
| ignore_index | Do not preserve indexes along concatenation axis, instead producing a new __range(total_length) index__ |

### Combining Data with Overlap

In [74]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [75]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [76]:
np.where(pd.isnull(a), b,a)  # performs the array-oriented equivalent of an if-else expression

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

Series has a __combine_first__ method, which performs the equivalent of __np.where__ operation along with panda's usual data alignement logic

In [77]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

With DataFrames, __combine_first__ does the same thing column by column, so you
can think of it as “patching” missing data in the calling object with data from the
object you pass

In [78]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [79]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [80]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting
- Basic operation for rearranging tabular data which is often referred as *reshape or pivot* operations

### Reshaping with Hierarchical Indexing
Hierarchical Indexing provides a consistent way to rearange data in a DF. There are two primary actions:
- __stack__ : This "rotates" or pivots from the columns in the data to the rows
- __unstack__ :  This pivots from the rows into the columns

In [81]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)), 
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [82]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [83]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with __stack__). You can unstack a different level by passing a level number or name

In [84]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [85]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [86]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [87]:
#unstacking might introduce missing data if all of the values in the level aren't found in each of the subgroups
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [88]:
#stacking filters out missing data by default making operation easily invertible
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [89]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [90]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [91]:
df.unstack(0)   #or df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [92]:
#can also indicate the name of the axis to stack
df.unstack(0).stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format
- A common way to store multiple time series in databases and CSV is in so-called __long or stacked__ format.
- __pivot__ is equivlent to creating hierarchical index using __set_index__ followed by a call to __unstack__

In [93]:
data = pd.read_csv('examples/macrodata.csv')
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008.0,3.0,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008.0,4.0,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009.0,1.0,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009.0,2.0,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [94]:
#PeriodIndex combines a year and quarter columns to create a kind of time interval
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [95]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')  #assigns end of the quarter date as an index
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [96]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


Above output is the so-called __long__ format for multiple time series, or other observational data with two or more keys. Each row in the table represents a single observation

<u>Data is frequently stored this way in relational databases like MySQL, as a fixed schema (column names and data types) allows the number of distinct values in the item column to change as data is added to the table.</u> In the previous example, __date__ and __item__ would usually be the primary keys (in relational database parlance), offering both relational integrity and easier joins. In some cases, the data may be more difficult to work with in this format; <u>you might prefer to have a DataFrame containing one column per distinct __item__ value indexed by timestamps in the __date__ column.</u> DataFrame’s __pivot__ method performs exactly this transformation:

In [97]:
#first two values passed are the columns to be used respectively as the row and column index and other value column to fill DF
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [98]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,1.293164
1,1959-03-31 23:59:59.999999999,infl,0.000,1.382213
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.218170
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.006940
4,1959-06-30 23:59:59.999999999,infl,2.340,0.028371
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.537586
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.793930
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.419020
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.325142
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-1.806401


In [99]:
#if value is omitted then DF with hierarchical columns is obtained
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.382213  1.293164   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.028371  0.006940   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.419020 -1.793930   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.800710 -1.806401   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.563268  0.323315   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.218170  
1959-06-30 23:59:59.999999999  0.537586  
1959-09-30 23:59:59.999999999 -0.325142  
1959-12-31 23:59:59.999999999 -0.385011  
1960-03-31 23:59:59.999999999 -0.694833

In [100]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [101]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')  #equivalent to pivot function
unstacked[:10]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.382213  1.293164   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.028371  0.006940   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.419020 -1.793930   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.800710 -1.806401   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.563268  0.323315   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.172150  0.578074   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.244366  1.273736   
1960-12-31 23:59:59.999999999  1.21  2802.616   6.3 -0.345988 -0.341524   
1961-03-31 23:59:59.999999999 -0.40  2819.264   6.8  0.744826  0.401486   
1961-06-30 23:59:59.999999999  1.47  2872.005   7.0  0.256549 -1.331471   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.218170  
1959-06-30 23:59:59.999999999  0.537586  
1959-09-30 23:59:59.999999999 -0.325142  
1959-12-31 23:59:59.999999999 -0.385011  
1960-03-31 23:59:59.999999999 -0.694833  
1960-06-30 23:59:59.999999999 -0.901081  
1960-09-30 23:59:59.999999999  0.949702  
1960-12-31 23:59:59.999999999 -0.181646  
1961-03-31 23:59:59.999999999 -0.046279  
1961-06-30 23:59:59.999999999  0.743611

### Pivoting "Wide" to "Long" Format
- Inverse operation to __pivot__ for DF is __pandas.melt__
- __melt__ function merges muliple columns into one, producing a DF that is longer than the input

In [102]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],'A': [1, 2, 3],'B': [4, 5, 6],'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


With __pandas.melt__, we must indicate which columns (if any) are group indicators. In this case, we have __key__ as a group indicator

In [103]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [104]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [105]:
#move the data back into a column
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [110]:
#specifying a subset of columns to use as value columns
pd.melt(df, id_vars=['key'], value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [108]:
#using without any group identifiers
pd.melt(df, value_vars=['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [111]:
pd.melt(df, value_vars=['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
